In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

C:\Users\Gamma\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def parse(link):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    request = requests.get(link, headers = headers)
    soup = BeautifulSoup(request.text, 'html.parser')
    return soup

In [3]:
items = []
for i in range(1, 4):
    newRequest = parse(f"https://www.thewhiskyexchange.com/c/33/american-whiskey?pg={i}&psize=24&sort=pasc")
    itemLinks = newRequest.findAll("a", attrs={'href': re.compile("^/p/")})
    temp = ""
    for item in itemLinks:
        temp = "https://www.thewhiskyexchange.com" + item["href"]
        items.append(temp)

In [ ]:
whiskey = []
for item in items[1:25]:
    product = {}
    newRequest = parse(item)
    try:
        newRequest.find("h1", {'class': "product-main__name"}).span.decompose()
    except:
        pass
    product["title"] = newRequest.find("h1", {'class': "product-main__name"}).text.strip()
    try:
        product["source"] = newRequest.find("ul", {'class': 'product-main__meta'}).text.strip()
    except:
        pass
    product["desc"] = newRequest.find('div', {'class': 'product-main__description'}).text.strip() 
    try:
        product["ratings_count"] = re.search("\d+\s", newRequest.find("span", {'class': 'review-overview__count'}).text.strip()).group().strip()
    except:
        None
        
    whiskey.append(product)

In [ ]:
whiskeyDF = pd.DataFrame(whiskey)

In [ ]:
whiskeyDF.to_csv("american-whiskey.csv", index = False)
print("Saved")